# Time Series Prediction with Reservoir Computing ($RCN$)

AUTHOR: Anurag Dutta (anuragdutta.research@gmail.com)

SUPERVISOR: Tanujit Chakraborty (tanujit.chakraborty@sorbonne.ae)

CREATED: 2023-03-07 19:58:34

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Gathering Dependencies

_Importing Required Libraries_

In [2]:
pip install hampel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import LSTM
import keras
import tensorflow as tf
from hampel import hampel
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

## Loading Datasets

_Bjørnøya_

In [4]:
data = pd.read_csv("/content/drive/MyDrive/new/datasets/Rainfall.csv", sep = ";")
training_set = data.set_index("Time(norwegian mean time)")
training_set = training_set.iloc[:,2]

## Preprocessing the Data

In [5]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(training_set, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

Streaming output truncated to the last 5000 lines.
[0.4 0.6 0.2] 4.5
[0.6 0.2 4.5] 0.4
[0.2 4.5 0.4] 0.6
[4.5 0.4 0.6] 1.5
[0.4 0.6 1.5] 4.1
[0.6 1.5 4.1] 2.2
[1.5 4.1 2.2] 1.0
[4.1 2.2 1. ] 0.6
[2.2 1.  0.6] 0.8
[1.  0.6 0.8] 0.6
[0.6 0.8 0.6] 4.5
[0.8 0.6 4.5] 0.2
[0.6 4.5 0.2] 0.0
[4.5 0.2 0. ] 0.0
[0.2 0.  0. ] 0.8
[0.  0.  0.8] 0.4
[0.  0.8 0.4] 0.0
[0.8 0.4 0. ] 0.2
[0.4 0.  0.2] 0.0
[0.  0.2 0. ] 0.0
[0.2 0.  0. ] 0.2
[0.  0.  0.2] 0.0
[0.  0.2 0. ] 0.0
[0.2 0.  0. ] 0.0
[0. 0. 0.] 0.4
[0.  0.  0.4] 1.4
[0.  0.4 1.4] 0.0
[0.4 1.4 0. ] 0.0
[1.4 0.  0. ] 0.0
[0. 0. 0.] 0.0
[0. 0. 0.] 0.0
[0. 0. 0.] 0.0
[0. 0. 0.] 0.0
[0. 0. 0.] 0.9
[0.  0.  0.9] 0.6
[0.  0.9 0.6] 0.0
[0.9 0.6 0. ] 0.2
[0.6 0.  0.2] 0.0
[0.  0.2 0. ] 8.0
[0.2 0.  8. ] 0.2
[0.  8.  0.2] 0.0
[8.  0.2 0. ] 0.0
[0.2 0.  0. ] 0.8
[0.  0.  0.8] 0.0
[0.  0.8 0. ] 0.4
[0.8 0.  0.4] 2.0
[0.  0.4 2. ] 3.3
[0.4 2.  3.3] 0.6
[2.  3.3 0.6] 0.0
[3.3 0.6 0. ] 0.2
[0.6 0.  0.2] 0.0
[0.  0.2 0. ] 0.8
[0.2 0.  0.8] 1.0
[0.  0.8 1. ]

## Model ($RCN$)

In [6]:
pip install reservoirpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from reservoirpy.nodes import Reservoir, Ridge

reservoir = Reservoir(units=100, lr=0.5, sr=0.001)
readout = Ridge(output_dim=3, ridge=1e-8)

In [8]:
esn = reservoir >> readout

In [9]:
esn.fit(X[:12253], X[1:12254], warmup=100)

Running Model-0:   0%|          | 0/1 [00:00<?, ?it/s]
Running Model-0: 199it [00:00, 1980.39it/s]           
Running Model-0: 398it [00:00, 1930.05it/s]
Running Model-0: 592it [00:00, 1918.21it/s]
Running Model-0: 879it [00:00, 2284.59it/s]
Running Model-0: 1193it [00:00, 2587.81it/s]
Running Model-0: 1564it [00:00, 2966.09it/s]
Running Model-0: 1933it [00:00, 3199.16it/s]
Running Model-0: 2278it [00:00, 3276.28it/s]
Running Model-0: 2607it [00:00, 3167.18it/s]
Running Model-0: 2925it [00:01, 2949.61it/s]
Running Model-0: 3224it [00:01, 2944.87it/s]
Running Model-0: 3571it [00:01, 3095.56it/s]
Running Model-0: 3933it [00:01, 3246.61it/s]
Running Model-0: 4300it [00:01, 3370.24it/s]
Running Model-0: 4642it [00:01, 3384.73it/s]
Running Model-0: 4982it [00:01, 3045.43it/s]
Running Model-0: 5326it [00:01, 3152.37it/s]
Running Model-0: 5693it [00:01, 3296.23it/s]
Running Model-0: 6028it [00:01, 3269.73it/s]
Running Model-0: 6359it [00:02, 3276.03it/s]
Running Model-0: 6700it [00:02, 3311.7

Fitting node Ridge-0...


'Model-0': Model('Reservoir-0', 'Ridge-0')

In [10]:
predictions = esn.run(X[12254:-1])

Running Model-0: 3062it [00:01, 2375.73it/s]


## Metrics ($RCN$)

In [11]:
import reservoirpy.observables
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = np.sqrt(mean_squared_error(X[12254:-1], predictions))
mae = np.sqrt(mean_absolute_error(X[12254:-1], predictions))
print('Test RMSE: %.3f' % rmse)
print('Test MAE: %.3f' % mae)

Test RMSE: 2.729
Test MAE: 1.214
